## The Monty Hall TV Show.

There are **3 doors**. Behind one door is a **car** 🚗. Behind the other two are **goats** 🐐🐐 (no prize).

1. You **pick a door** — say, Door #1.
2. The host (who *knows* what's behind every door) opens **one of the other doors** to reveal a goat.
3. The host then asks: **"Do you want to switch to the other door?"**

*You can keep your pick, or choose to switch*

What would you do?

## How the Simulation Works

We'll write a program that plays the game **hundreds of times** and counts how often each strategy wins.

```
Strategy A — Always Stay:    Keep your original door.
Strategy B — Always Switch:  Always switch to the other door.
```

By running it many times, the *true probability* emerges from the noise.

In [18]:
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sns.set_theme(style="ticks", font_scale=1.2)

def run_monty_hall(num_trials: int, switch_door: bool) -> int:
    """Play the Monty Hall game num_trials times and return total wins."""
    wins = 0
    doors = [0, 1, 2]

    for _ in range(num_trials):
        prize_door   = random.choice(doors)
        player_pick  = random.choice(doors)

        # Monty opens a door that is neither the prize nor the player's pick
        monty_opens  = random.choice(
            [d for d in doors if d != prize_door and d != player_pick]
        )

        if switch_door:
            final = [d for d in doors if d != player_pick and d != monty_opens][0]
        else:
            final = player_pick

        if final == prize_door:
            wins += 1

    return wins

Use the slider below to choose how many games to simulate — the chart updates automatically.

In [ ]:
@widgets.interact(n=widgets.IntSlider(
    value=100, min=100, max=1000, step=100,
    description="# Games:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="500px"),
))
def plot_monty_hall(n):
    wins_stay   = run_monty_hall(n, switch_door=False)
    wins_switch = run_monty_hall(n, switch_door=True)

    df = pd.DataFrame({
        "Strategy": ["Always Stay", "Always Switch"],
        "Wins":     [wins_stay, wins_switch],
        "Win %":    [round(wins_stay / n * 100, 1), round(wins_switch / n * 100, 1)]
    })

    fig, ax = plt.subplots(figsize=(7, 5))
    sns.barplot(data=df, x="Strategy", y="Wins",
                hue="Strategy", dodge=False, legend=False,
                palette=["#E8575A", "#5B8FB9"],
                edgecolor="white", linewidth=1.5, width=0.5, ax=ax)

    ax.set_title(f"Monty Hall Results — {n:,} Games per Strategy",
                 fontsize=14, weight="bold", pad=12)
    ax.set_ylabel("Number of Wins")
    ax.set_xlabel("")
    ax.set_ylim(0, n * 1.1)

    for i, container in enumerate(ax.containers):
        row = df.iloc[i]
        ax.bar_label(container,
                     labels=[f"{row['Wins']} wins\n({row['Win %']}%)"],
                     fontsize=12, fontweight="bold", padding=6)

    ax.axhline(n * 1/3, color="#E8575A", linestyle="--", linewidth=1,
               label=f"Theory: Stay = 33.3% ({n//3} wins)")
    ax.axhline(n * 2/3, color="#5B8FB9", linestyle="--", linewidth=1,
               label=f"Theory: Switch = 66.7% ({n*2//3} wins)")
    ax.legend(fontsize=10)

    sns.despine(left=True, bottom=True)
    plt.tight_layout()
    plt.show()

    print(f"\nStay win rate:   {wins_stay/n*100:.1f}%  (theory: 33.3%)")
    print(f"Switch win rate: {wins_switch/n*100:.1f}%  (theory: 66.7%)")

interactive(children=(IntSlider(value=100, description='# Games:', layout=Layout(width='500px'), max=1000, min…

## Let's revisit the game

The math is surprisingly simple once you see it:

| Your initial pick | Prize location | Switch result |
|:-----------------:|:--------------:|:-------------:|
| ✅ Correct (1/3 chance) | Same door | **Lose** |
| ❌ Wrong (2/3 chance)   | Other door | **Win** |

When you *stay*, you only win if your **first guess was right** → 1/3 chance.

When you *switch*, you win if your **first guess was wrong** → 2/3 chance.

The host's action of opening a goat door gives you **information** — switching lets you exploit it.

---
It's a real TV show, and real game BTW. Someone in the 90s found out you can win 2/3 of the time!!!